In [1]:
import os, json, time, datetime as dt ,csv, pathlib
import requests
import pandas as pd
from typing import List, Dict
from bs4 import BeautifulSoup
from dotenv import load_dotenv

raw_data =pathlib.Path("data/raw")
raw_data.mkdir(parents = True, exist_ok = True)

load_dotenv()

/opt/anaconda3/envs/bootcamp_env/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


False

In [2]:
def safe_stamp():
    return dt.datetime.now().strftime("%Y%m%d-%H%M%S")

def safe_filename(prefix: str, meta: Dict[str, str]) -> str:
    mid = "_".join([f"{k}-{str(v).replace(' ', '-')[:20]}" for k, v in meta.items()])
    return f"{prefix}_{mid}_{safe_stamp()}.csv"

def validate_df(df: pd.DataFrame, required_cols: List[str], dtypes_map: Dict[str, str]) -> Dict[str, str]:
    msgs = {}
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        msgs['missing_cols'] = f"Missing columns: {missing}"
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                if dtype == 'datetime64[ns]':
                    pd.to_datetime(df[col])
                elif dtype == 'float':
                    pd.to_numeric(df[col])
            except Exception as e:
                msgs[f'dtype_{col}'] = f"Failed to coerce {col} to {dtype}: {e}"
    na_counts = df.isna().sum().sum()
    msgs['na_total'] = f"Total NA values: {na_counts}"
    return msgs

In [3]:
os.environ["ALPHAVANTAGE_API_KEY"] = "2TMJOG8YVAAWX7IV"
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded ALPHAVANTAGE_API_KEY?",bool(ALPHA_KEY))

Loaded ALPHAVANTAGE_API_KEY? True


In [4]:

# --- 2. API Pull Function ---
def pull_alphavantage_timeseries(
    ticker: str,
    function: str = 'TIME_SERIES_DAILY',
    outputsize: str = 'full',
    api_key: str = ALPHA_KEY,
) -> pd.DataFrame:
    """
    Pulls time series data from Alpha Vantage, handles errors,
    and returns a DataFrame.
    """
    if not api_key:
        raise ValueError("Alpha Vantage API key not found. Please set it in your .env file.")

    url = "https://www.alphavantage.co/query"
    params = {
        "function": function,
        "symbol": ticker,
        "outputsize": outputsize,
        "apikey": api_key,
    }

    try:
        print(f"Fetching {function} data for {ticker}...")
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return pd.DataFrame()

    if 'Error Message' in data:
        print(f"API returned an error: {data['Error Message']}")
        return pd.DataFrame()
    
    time_series_key = "Time Series (Daily)"
    time_series = data.get(time_series_key)
    
    if not time_series:
        print("No time series data found in the response.")
        return pd.DataFrame()
    
    df = pd.DataFrame.from_dict(time_series, orient='index')
    return df
    
df = pull_alphavantage_timeseries(ticker='AAPL', api_key=ALPHA_KEY)
print(df)

Fetching TIME_SERIES_DAILY data for AAPL...
             1. open   2. high    3. low  4. close 5. volume
2025-08-20  229.9800  230.4700  225.7700  226.0100  42089146
2025-08-19  231.2750  232.8700  229.3500  230.5600  39402564
2025-08-18  231.7000  233.1200  230.1100  230.8900  37476188
2025-08-15  234.0000  234.2800  229.3350  231.5900  56038657
2025-08-14  234.0550  235.1200  230.8500  232.7800  51916275
...              ...       ...       ...       ...       ...
1999-11-05   84.6200   88.3700   84.0000   88.3100   3721500
1999-11-04   82.0600   85.3700   80.6200   83.6200   3384700
1999-11-03   81.6200   83.2500   81.0000   81.5000   2932700
1999-11-02   78.0000   81.6900   77.3100   80.2500   3564600
1999-11-01   80.0000   80.6900   77.3700   77.6200   2487300

[6490 rows x 5 columns]


In [5]:
if __name__ == "__main__":
    # Define the URL for the S&P 500 Wikipedia page
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    print(f"Fetching data from: {url}")
    
    try:
        # Request the HTML content of the page
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the main table containing the S&P 500 components
        # The table has the CSS class 'wikitable' and 'sortable'
        sp500_table = soup.find('table', {'class': 'wikitable'})
        
        if not sp500_table:
            raise ValueError("Could not find the S&P 500 table on the page.")
        
        # Extract headers from the table
        headers = [th.text.strip() for th in sp500_table.find_all('th')]
        
        # Extract data rows
        data_rows = []
        for row in sp500_table.find_all('tr')[1:]:  # Skip the header row
            cols = row.find_all('td')
            # Extract text from each cell and strip whitespace
            cols = [ele.text.strip() for ele in cols]
            data_rows.append(cols)
            
        # Create a pandas DataFrame from the extracted data and headers
        df = pd.DataFrame(data_rows, columns=headers)
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        exit()
    except ValueError as e:
        print(f"Parsing error: {e}")
        exit()
        
    # --- Data Cleaning and Validation ---
    print("\n--- Raw DataFrame Info ---")
    print(df.info())
    print("\nFirst 5 rows of raw DataFrame:")
    print(df.head())

    # Drop the 'SEC filings' column as it's not needed for the core data
    if 'SEC filings' in df.columns:
        df = df.drop(columns=['SEC filings'])
    
    # Define the columns we want to keep and their data types
    required_cols = ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added']
    dtype_map = {
        'Symbol': 'string',
        'Security': 'string',
        'GICS Sector': 'string',
        'GICS Sub-Industry': 'string',
        'Headquarters Location': 'string',
        'Date added': 'datetime64[ns]'
    }
    
    # Check for presence of required columns
    if not set(required_cols).issubset(df.columns):
        print(f"\nMissing required columns. Found: {df.columns.tolist()}")
        exit()
        
    # Apply the data types and perform validation
    df['Date added'] = pd.to_datetime(df['Date added'], errors='coerce')
    validation_msgs = validate_df(df, required_cols, dtype_map)
    
    print("\n--- Validation Report ---")
    if validation_msgs:
        for key, msg in validation_msgs.items():
            print(f"- {key}: {msg}")
    else:
        print("✅ Data validation successful. No issues found.")

    print("\n--- Final DataFrame Summary ---")
    print(f"Shape: {df.shape}")
    print("Data Types:\n", df.dtypes)
    print("\nHead of Final DataFrame:")
    print(df.head())

    # --- Save Raw Data ---
    meta_info = {'source': 'wikipedia', 'content': 'sp500-components'}
    file_path = DATA_RAW / safe_filename('web_scrape', meta_info)
    df.to_csv(file_path, index=False)
    print(f"\nSuccessfully saved raw data to {file_path}")


Fetching data from: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

--- Raw DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    object
 7   Founded                503 non-null    object
dtypes: object(8)
memory usage: 31.6+ KB
None

First 5 rows of raw DataFrame:
  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories  

NameError: name 'DATA_RAW' is not defined